Reference- Last assignment+https://www.tensorflow.org/tutorials/text/text_generation

In [ ]:
import tensorflow as tf
import numpy as np
import keras

In [ ]:
!python prepare_data2.py shk_input.txt skp \\n -m 500

2020-12-03 23:01:38.387300: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Split input into 40000 sequences...
Longest sequence is 65 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
40000 sequences remaining.
Longest remaining sequence has length 65.
Removing length-0 sequences...
40000 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
S

In [ ]:
#Getting the data
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(parse_seq) #since changing seq lengths

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'F': 3, 'h': 4, 'O': 5, 's': 6, 'e': 7, 'c': 8, 'A': 9, 'N': 10, 'I': 11, 'W': 12, '3': 13, 'w': 14, 't': 15, 'H': 16, '!': 17, 'p': 18, '$': 19, ',': 20, 'U': 21, 'l': 22, 'M': 23, 'E': 24, 'Z': 25, 'z': 26, 'L': 27, 'G': 28, 'n': 29, 'q': 30, 'v': 31, '?': 32, 'T': 33, 'Y': 34, 'u': 35, '-': 36, ':': 37, 'J': 38, 'D': 39, ' ': 40, 'o': 41, 'K': 42, 'a': 43, 'B': 44, 'm': 45, 'R': 46, 'd': 47, '&': 48, 'X': 49, 'S': 50, 'g': 51, '\n': 52, "'": 53, 'j': 54, 'Q': 55, 'b': 56, 'P': 57, 'V': 58, '.': 59, 'k': 60, 'f': 61, 'r': 62, 'i': 63, 'x': 64, ';': 65, 'y': 66, 'C': 67, '<PAD>': 0, '<S>': 1, '</S>': 2}
68


In [ ]:
#for current and expected time stamps
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

data = data.map(split_input_target)

In [ ]:

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = data.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, drop_remainder=True) #to make the batch size of eqal since sequence is of variable lengths

In [ ]:
def build_model(vocab_size, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, 256, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=68,
    rnn_units=512,
    batch_size=128)

In [ ]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (128, None, 256)          17408     
_________________________________________________________________
lstm_27 (LSTM)               (128, None, 512)          1574912   
_________________________________________________________________
dense_27 (Dense)             (128, None, 68)           34884     
Total params: 1,627,204
Trainable params: 1,627,204
Non-trainable params: 0
_________________________________________________________________


In [ ]:

#running it
epoch = 50
opt = tf.keras.optimizers.Adam()
for e in range(epoch):
  for batch_num, (batch_data,y) in enumerate(data): #this is already batched of 128 batch size
    total_loss = 0.0 #total loss over sequence 
    
    with tf.GradientTape(persistent=True) as tape:
      actual_chars = tf.TensorArray(tf.int64, size=BATCH_SIZE)
      for char_pos, which_seq in enumerate(batch_data): #goign over the sequence in each batch
            actual_chars = actual_chars.write(char_pos, tf.math.count_nonzero(which_seq))
      mask = tf.sequence_mask(actual_chars.stack(), dtype=tf.float32)
      logits = model(batch_data)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
      loss = loss * mask
        
    grads = tape.gradient(loss, model.trainable_variables) #gradients against the parametes/weights and biasas
    opt.apply_gradients(zip(grads, model.trainable_variables))
    total_loss = tf.reduce_sum(loss)/float(tf.reduce_sum(actual_chars.stack())) # this is total loss over the whole batch
    if batch_num % 100 == 0:
          print('Epoch {} Batch {} Loss {}'.format(e, batch_num, total_loss))
    

  model.reset_states()

Epoch 0 Batch 0 Loss 4.220046043395996
Epoch 0 Batch 100 Loss 2.468925714492798
Epoch 0 Batch 200 Loss 2.22340726852417
Epoch 0 Batch 300 Loss 1.9901728630065918
Epoch 1 Batch 0 Loss 2.4844579696655273
Epoch 1 Batch 100 Loss 1.8927117586135864
Epoch 1 Batch 200 Loss 1.756066918373108
Epoch 1 Batch 300 Loss 1.767006516456604
Epoch 2 Batch 0 Loss 1.8924732208251953
Epoch 2 Batch 100 Loss 1.6891064643859863
Epoch 2 Batch 200 Loss 1.6457432508468628
Epoch 2 Batch 300 Loss 1.621482253074646
Epoch 3 Batch 0 Loss 1.6948469877243042
Epoch 3 Batch 100 Loss 1.5173479318618774
Epoch 3 Batch 200 Loss 1.5612457990646362
Epoch 3 Batch 300 Loss 1.503048300743103
Epoch 4 Batch 0 Loss 1.5808364152908325
Epoch 4 Batch 100 Loss 1.4707497358322144
Epoch 4 Batch 200 Loss 1.5193747282028198
Epoch 4 Batch 300 Loss 1.4869189262390137
Epoch 5 Batch 0 Loss 1.4968221187591553
Epoch 5 Batch 100 Loss 1.3834106922149658
Epoch 5 Batch 200 Loss 1.3893227577209473
Epoch 5 Batch 300 Loss 1.455060362815857
Epoch 6 Batch

In [ ]:
"""tf.saved_model.save(model, "/content/drive/MyDrive/lstm_folder")"""

'tf.saved_model.save(model, "/content/drive/MyDrive/lstm_folder")'

In [ ]:
model.save_weights('/content/drive/MyDrive/lstm_folder/my_model_weights.h5')

In [ ]:
model_p = build_model(
    vocab_size=68,
    rnn_units=512,
    batch_size=1)
model_p.load_weights("/content/drive/MyDrive/lstm_folder/my_model_weights.h5")

model_p.build(tf.TensorShape([1, None]))

model_p.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (1, None, 256)            17408     
_________________________________________________________________
lstm_29 (LSTM)               (1, None, 512)            1574912   
_________________________________________________________________
dense_29 (Dense)             (1, None, 68)             34884     
Total params: 1,627,204
Trainable params: 1,627,204
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
idx2char

array({'F': 3, 'h': 4, 'O': 5, 's': 6, 'e': 7, 'c': 8, 'A': 9, 'N': 10, 'I': 11, 'W': 12, '3': 13, 'w': 14, 't': 15, 'H': 16, '!': 17, 'p': 18, '$': 19, ',': 20, 'U': 21, 'l': 22, 'M': 23, 'E': 24, 'Z': 25, 'z': 26, 'L': 27, 'G': 28, 'n': 29, 'q': 30, 'v': 31, '?': 32, 'T': 33, 'Y': 34, 'u': 35, '-': 36, ':': 37, 'J': 38, 'D': 39, ' ': 40, 'o': 41, 'K': 42, 'a': 43, 'B': 44, 'm': 45, 'R': 46, 'd': 47, '&': 48, 'X': 49, 'S': 50, 'g': 51, '\n': 52, "'": 53, 'j': 54, 'Q': 55, 'b': 56, 'P': 57, 'V': 58, '.': 59, 'k': 60, 'f': 61, 'r': 62, 'i': 63, 'x': 64, ';': 65, 'y': 66, 'C': 67, '<PAD>': 0, '<S>': 1, '</S>': 2},
      dtype=object)

In [ ]:
def generate_text(model_p, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model_p.reset_states()
    for i in range(num_generate):
        predictions = model_p(input_eval)
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        #print("predicted_id",predicted_id)

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        #print("input_eval",input_eval)
        #print(ind_to_ch[predicted_id])

        text_generated.append(ind_to_ch[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model_p, start_string=u"ROMEO: "))

ROMEO:  I beseech you on, then both lost</S>ly</S>on!</S></S>oke</S>ge</S>!</S>el</S>.</S></S>well,--</S>essen,</S>velous,</S>able!</S></S></S>lough!</S>'</S></S>les,</S>eve</S></S>ign</S>s</S>:</S>ether.</S></S>ly a</S>t,</S></S>age!</S></S>il</S>!</S></S>ight,</S>absky,</S>!</S>eir up! or want your height</S>en</S>ing,</S></S>ign!</S></S>tituge.</S></S>'</S></S></S>ent,</S>and</S>ow</S>ly</S>el:</S></S>wage!</S></S>igh</S>!</S></S>il'd,</S>rait</S>er</S>.</S></S>wel</S></S></S></S>el</S>;</S></S>wed</S>!</S>curs,</S>cate to instruct her</S>ines</S>.</S></S>ings</S>ing</S>t</S>e!</S></S></S>ing,</S></S>lowsed</S>y</S>!</S>e</S>ge</S></S>ess,</S>'</S>!</S>ereth.</S></S>ere</S>le</S>ters</S>alled,</S>hatiest</S>ward.</S></S>y</S>t</S>ig</S>!</S></S>ote,</S></S>ace</S>ol!</S>but,</S>'</S></S>ignsting</S>t</S>Mentain!</S>-Notens.</S>,</S>ever'd.</S></S>kering, as four lovers,</S>herance.</S></S>y</S>s</S>ing</S>ing</S></S>ins!</S></S>e, but</S>fired</S>y,</S>'d!</S>ermorade.</S></S></S>in